In [11]:
import pandas as pd
import configparser
import pymysql
from sqlalchemy import create_engine
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_curve
import numpy as np
from scipy import stats
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

import warnings; warnings.simplefilter('ignore')

In [2]:
reader = Reader(rating_scale=(1,5))

In [3]:
parser = configparser.ConfigParser()
parser.read("config\mysql.conf")
hostname = parser.get("mysql_config", "hostname")
username = parser.get("mysql_config", "username")
password = parser.get("mysql_config", "password")
dbname = parser.get("mysql_config", "database")
port = parser.get("mysql_config", "port")

conn_str = 'mysql+pymysql://' + username + ':' + password + '@' + hostname + '/' + dbname
db_connection = create_engine(conn_str)

In [4]:
ratings = pd.read_sql('SELECT * FROM movies_dwh.v_ml_ratings', con = db_connection)

In [5]:
data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']], reader)
#data.split(n_folds=5)

In [8]:
svd = SVD()
#evaluate(svd, data, measures=['RMSE', 'MAE'])
cross_validate(svd, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8963  0.8916  0.8980  0.9091  0.8843  0.8959  0.0081  
MAE (testset)     0.6901  0.6880  0.6913  0.7023  0.6790  0.6901  0.0074  
Fit time          0.84    0.89    0.83    0.87    0.85    0.86    0.02    
Test time         0.12    0.19    0.12    0.19    0.12    0.15    0.03    


{'test_rmse': array([0.89634061, 0.89163909, 0.89799021, 0.90906842, 0.88432933]),
 'test_mae': array([0.69005868, 0.68804822, 0.69126153, 0.70225597, 0.67895998]),
 'fit_time': (0.840001106262207,
  0.8930330276489258,
  0.8319966793060303,
  0.869999885559082,
  0.8520009517669678),
 'test_time': (0.1210026741027832,
  0.18996667861938477,
  0.12399983406066895,
  0.1900022029876709,
  0.11900091171264648)}

In [9]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [8]:
ratings[ratings['user_id'] == 1]

,user_id,movie_id,rating,rating_date,rating_time
0,1,31,2.5,2009-12-14,0 days 03:52:24
1,1,1029,3.0,2009-12-14,0 days 03:52:59
2,1,1061,3.0,2009-12-14,0 days 03:53:02
3,1,1129,2.0,2009-12-14,0 days 03:53:05
4,1,1172,4.0,2009-12-14,0 days 03:53:25
5,1,1263,2.0,2009-12-14,0 days 03:52:31
6,1,1287,2.0,2009-12-14,0 days 03:53:07
7,1,1293,2.0,2009-12-14,0 days 03:52:28
8,1,1339,3.5,2009-12-14,0 days 03:52:05
9,1,1343,2.0,2009-12-14,0 days 03:52:11


In [9]:
svd.predict(1, 858, 5)

Prediction(uid=1, iid=858, r_ui=5, est=3.7496083745224236, details={'was_impossible': False})

Dla każdego użytkownika wśród aktywnych użytkowników
    sprawdź jakich filmów nie ocenił
    dla nieocenionych filmów dokonaj predykcji
    zostaw 5 najwyżej ocenionych filmów
    wynik zapisz do dataframe
    dołącz dataframe do dataframe zbiorczego

In [13]:
def predict_rating(movie_id):
    return svd.predict(user_id, movie_id).est

In [11]:
users = ratings['user_id'].unique()
movies = set(ratings['movie_id'].unique())
movies_predictions_df = pd.DataFrame(columns=['movie_id', 'user_id', 'predicted_rating'])
for user_id in users:
    movies_rated = set(ratings[ratings['user_id']==user_id]['movie_id'].to_list())
    movies_unrated = list(movies - movies_rated)
    movies_predictions = pd.DataFrame(movies_unrated)
    movies_predictions['user_id'] = user_id
    movies_predictions['predicted_rating'] = movies_predictions[0].apply(predict_rating)
    movies_predictions_df = pd.concat([movies_predictions_df, movies_predictions], axis=0)

KeyboardInterrupt: 

In [28]:
users = ratings['user_id'].unique()
movies = set(ratings['movie_id'].unique())
movies_predictions_df = pd.DataFrame(columns=['movie_id', 'user_id', 'predicted_rating'])
user_id = 1
movies_rated = set(ratings[ratings['user_id']==user_id]['movie_id'].to_list())
movies_unrated = list(movies - movies_rated)
movies_predictions = pd.DataFrame(movies_unrated, columns=['movie_id'])
movies_predictions['user_id'] = user_id
movies_predictions['predicted_rating'] = movies_predictions[0].apply(predict_rating)
movies_predictions_df = pd.concat([movies_predictions_df, movies_predictions], axis=0)

KeyError: 0

In [14]:
movies_rated = set(ratings[ratings['user_id'] == user_id]['movie_id'].to_list())
movies_unrated = list(movies - movies_rated)
movies_predictions = pd.DataFrame(movies_unrated, columns=['movie_id'])
movies_predictions['user_id'] = user_id
movies_predictions['predicted_rating'] = movies_predictions['movie_id'].apply(predict_rating)
movies_predictions_df = pd.concat([movies_predictions_df, movies_predictions], axis=0)

movies_predictions_df

NameError: name 'user_id' is not defined

Walidacja

In [20]:
val = ratings[['user_id', 'movie_id', 'rating']]
val['predicted_rating'] = val.apply(lambda row: svd.predict(row['user_id'], row['movie_id']).est, axis=1)
val

,user_id,movie_id,rating,predicted_rating
0,1,31,2.5,2.375360
1,1,1029,3.0,2.792272
2,1,1061,3.0,2.741100
3,1,1129,2.0,2.451734
4,1,1172,4.0,3.648309
...,...,...,...,...
99999,671,6268,2.5,3.289681
100000,671,6269,4.0,3.814316
100001,671,6365,4.0,3.645616
100002,671,6385,2.5,3.556503


In [16]:
y_true = val['rating']
y_pred = val['predicted_rating']

# Obliczanie RMSE
rmse = sqrt(mean_squared_error(y_true, y_pred))
print(f'RMSE: {rmse}')

# Obliczanie MAE
mae = mean_absolute_error(y_true, y_pred)
print(f'MAE: {mae}')


RMSE: 0.6412155135075186
MAE: 0.4964527251208034
